# Her skal vi undersøke dataen hentet og bruke Pandas/SQL for å forstå strukturen

### Her begynner vi med å forstå dataen fra frost bedre, og gjøre det om til en pandas dataframe, slik at det blir lett å jobbe med videre. 

I tillegg skal vi se på hvordan vi kan bruke pandasql (sqldf) for å gjøre querys på dataen og filtrere gjennom lettvint. Vi begynner med å hente file_path

In [13]:
import pandas as pd
from pandasql import sqldf
import os

# Henter file path

notebook_directory = os.getcwd()
root = os.path.abspath(os.path.join(notebook_directory, ".."))
file_path = os.path.join(root, "data", "Frost_Observations.json")

Dataen vi hentet er laget med mange dictionaries, som igjen inneholder flere dictionaries. For å kun få ut relevant data kan vi filtrere gjennom python for enkelte "entries" i forskjellige dictionaries. Deretter kan vi gjøre det om til en dataframe ved bruk av pandas og se hvordan dataen ser ut

In [ ]:
import json

with open(file_path,"r") as fil: #leser filen og lagrer den som "data"
    data=json.load(fil)

# filtrerer dataen for bare det som er relevant til å analysere
brukbar_data = [
    {
        "elementId": obs["elementId"],
        "sourceId": entry["sourceId"],
        "referenceTime": entry["referenceTime"],
        "value": obs["value"],
        "unit": obs["unit"]
    }
    for entry in data["data"]
    for obs in entry["observations"] 
]

df=pd.DataFrame(brukbar_data) #Dataframe

pd.set_option('display.max_columns', None) #justerer output display
pd.set_option('display.width', 1000)
print(df.head())

Da har vi fått lagd en dataframe med pandas som viser relevant data. Det ser litt uryddig ut og det passer bedre å ha hver observasjon som kolonner og så verdiene nedover, så vi kan bruke pd.pivot til til å gjøre det mer ryddig. 

Men det at vi har 2 verdier for mean(air_temperature P1D) og for sum(precipitation_amount P1D) gjør at dette ikke fungerer. Foreløpig kan vi kombinere det ved å ta gjennomsnittsverdien av de 2 målte, slik at vi kan visualisere dataframen bedre. Vi kan bruke "groupby" funksjonen for å samle de 2 identiske elementene og slå sammen verdiene med gjennomsnittet. 

In [ ]:
rader=[]

# henter relevant data ved å iterere gjennom i forskjellige dictionaries
for entry in data["data"]:
    referenceTime = entry["referenceTime"]
    for obs in entry["observations"]:
         rader.append({
            "referenceTime": referenceTime,
            "elementId": obs["elementId"],
            "value": obs["value"],
        })
         
df=pd.DataFrame(rader) #dataframe

# slår sammen der det er 2 av mean(air_temperature P1D) og tar gjennomsnittet, 
# gjør pivot av dataframe, slik at kolonnene blir type observasjon

df_gruppering = df.groupby(["referenceTime", "elementId"], as_index=False)["value"].mean()
df2=df_gruppering.pivot(index="referenceTime",columns="elementId",values="value")
df2=df2.reset_index()
print(df2.head())



In [ ]:
# lagrer df2 som json fil for videre bruk
df2.to_json('../data/df2_data.json',indent=4)

Her ser vi dataen på en mer forståelig måte. Det er verdt å merke at "boolean_clear_sky_weather(cloud_area_fraction P1D)" er boolean verdier, så i datahåndteringen kan vi ta hensyn til det og omgjøre det fra float. 

## Bruker PANDAS SQL (sqldf) til å hente data fra DataFrame

Nå som vi har en fungerende DataFrame, kan vi hente diverse type data og spesifikk data ved bruk av Pandas SQL/sqldf: 

1) Gjennomsnittlig temperatur for Juli i årene tilgjengelig
2) Alle dager der 24 timers gjennomsnittstemperaturen er over 20 grader og der det ikke var skyete basert på "boolean_clear_sky_weather..."

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

query = """

    SELECT strftime('%Y',referenceTime) AS År,
    AVG("mean(air_temperature P1D)") AS avg_temperatur_juli
    FROM df2
    WHERE strftime('%m',referenceTime) = '07'
    GROUP BY År
"""
resultat=pysqldf(query)
print(resultat)

Her har vi for hele tidsperioden vi har i flere år:

In [ ]:
query = """
    SELECT strftime('%Y-%m', referenceTime) AS Month, 
           AVG("mean(air_temperature P1D)") AS avg_temperature
    FROM df2
    GROUP BY Month
    ORDER BY Month;
"""
result = sqldf(query, globals())
print(result)

In [ ]:
query2="""
    SELECT * FROM df2
    WHERE "mean(air_temperature P1D)">=20.0
    ORDER BY "mean(air_temperature P1D)" DESC;
"""
resultat=pysqldf(query2)
print(resultat)

## Omgjøring av luftkvalitet-data til panda dataframe + forståelse: 

Her skal vi sortere gjennom luftkvalitetsdataen. Vi henter file-path og gjør om til data frame med pandas

In [ ]:
import pandas as pd
from pandasql import sqldf
import os

# Henter file path

notebook_directory = os.getcwd()
root = os.path.abspath(os.path.join(notebook_directory, ".."))
file_path_L = os.path.join(root, "data", "luftkvalitet_Kirkeveien_all_years.json")


Først kan vi hente generell informasjon om det som ligger i filen, og lager en dataframe basert på geneerell info, som stasjon, område, hva vi måler og id: 

In [ ]:
import json

with open(file_path_L,"r") as fil: #leser filen og lagrer den som "data"
    luftdata=json.load(fil)

general_data = []
for entry in luftdata:
    # Extract general information
    general_info = {
        "id": entry.get("id"),
        "zone": entry.get("zone"),
        "municipality": entry.get("municipality"),
        "area": entry.get("area"),
        "station": entry.get("station"),
        "component": entry.get("component"),
        "unit": entry.get("unit"),
        "latitude": entry.get("latitude"),
        "longitude": entry.get("longitude"),
    }
    
    # Extract values (nested list of dictionaries)
    for value in entry.get("values", []):
        # Combine general info with each value
        row = general_info.copy()
        row.update(value)  # Add fields from the "values" dictionary
        general_data.append(row)

# Create a DataFrame
df1 = pd.DataFrame(general_data)

# Display the first few rows
print(df1.head())

In [ ]:
processed_data = []
for entry in luftdata:
    component = entry.get("component")  # e.g., NO2, PM10, etc.
    for value_entry in entry.get("values", []):
        # Extract date and value
        processed_data.append({
            "date": value_entry.get("dateTime"),  # Assuming "fromTime" is the date field
            "component": component,
            "value": value_entry.get("value")  # Assuming "value" is the observed value
        })

# Create a DataFrame
df = pd.DataFrame(processed_data)

# Pivot the DataFrame to have components as columns
pivot_df = df.pivot(index="date", columns="component", values="value")

# Reset the index to make it easier to work with
pivot_df.reset_index(inplace=True)
pivot_df["date"] = pd.to_datetime(pivot_df["date"], errors="coerce").dt.tz_localize(None)

print(pivot_df.head())

In [ ]:
# Lagrer filen som "df1_data.json"

pivot_df.to_json('../data/df1_data.json',indent=4)

Her er et eksempel på bruk av sqldf for å hente data fra dataframe om luftkvalitet: 

- Total gjennomsnittlig verdi for de forskjellige partikkelmålingene

In [ ]:
query = """
    SELECT 
        AVG("NO2") AS avg_NO2,
        AVG("PM10") AS avg_PM10,
        AVG("PM2.5") AS avg_PM25,
        AVG("CO") AS avg_CO,
        AVG("NO") AS avg_NO,
        AVG("NOx") AS avg_NOx
    FROM pivot_df
"""
result = pysqldf(query)
print(result)